# Example: Calibration
----------------------

This example shows how to calibrate a classifier through atom.

The data used is a variation on the [Australian weather dataset](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package) from Kaggle. You can download it from [here](https://github.com/tvdboom/ATOM/blob/master/examples/datasets/weatherAUS.csv). The goal of this dataset is to predict whether or not it will rain tomorrow training a binary classifier on target `RainTomorrow`.

## Load the data

In [1]:
# Import packages
import pandas as pd
from atom import ATOMClassifier

In [2]:
# Load the data
X = pd.read_csv("./datasets/weatherAUS.csv")

# Let's have a look
X.head()

## Run the pipeline

In [3]:
atom = ATOMClassifier(X, "RainTomorrow", n_rows=1e4, verbose=1, warnings=False)

# Apply data cleaning steps
atom.clean()
atom.impute(strat_num="median", strat_cat="most_frequent")
atom.encode(strategy="target", max_onehot=5, infrequent_to_value=0.05)

# Train a linear SVM
atom.run("gnb")

## Analyze the results

In [4]:
# Check the model's calibration
atom.plot_calibration()

In [5]:
# Let's try to improve it using the calibrate method
atom.winner.calibrate(method="isotonic", cv=5)

In [6]:
# And check again...
atom.plot_calibration()